# colors

> Color system builders for Tailwind CSS utilities

In [ ]:
#| default_exp builders.colors

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Union, Optional, Any
from enum import Enum
from cjm_fasthtml_tailwind.core.base import (
    BaseUtility, StandardUtility, BaseFactory,
    TailwindValue, is_custom_property, is_arbitrary_value
)

## Color Enums

Define enums for the 22 Tailwind color families and 11 shade values:

In [ ]:
#| export
class ColorFamily(str, Enum):
    """Tailwind CSS color families."""
    RED = "red"
    ORANGE = "orange"
    AMBER = "amber"
    YELLOW = "yellow"
    LIME = "lime"
    GREEN = "green"
    EMERALD = "emerald"
    TEAL = "teal"
    CYAN = "cyan"
    SKY = "sky"
    BLUE = "blue"
    INDIGO = "indigo"
    VIOLET = "violet"
    PURPLE = "purple"
    FUCHSIA = "fuchsia"
    PINK = "pink"
    ROSE = "rose"
    SLATE = "slate"
    GRAY = "gray"
    ZINC = "zinc"
    NEUTRAL = "neutral"
    STONE = "stone"

In [ ]:
#| export
class ColorShade(str, Enum):
    """Tailwind CSS color shade values."""
    SHADE_50 = "50"
    SHADE_100 = "100"
    SHADE_200 = "200"
    SHADE_300 = "300"
    SHADE_400 = "400"
    SHADE_500 = "500"
    SHADE_600 = "600"
    SHADE_700 = "700"
    SHADE_800 = "800"
    SHADE_900 = "900"
    SHADE_950 = "950"

## Special Color Values

Define special color values that are available in addition to the color families:

In [ ]:
#| export
SPECIAL_COLORS = {
    "inherit": "inherit",
    "current": "current", 
    "transparent": "transparent",
    "black": "black",
    "white": "white"
}

## Color Value Types

Define types for color values:

In [ ]:
#| export
# Type for a complete color specification (e.g., "red-500", "blue-300")
ColorSpec = Union[str, tuple[Union[ColorFamily, str], Union[ColorShade, str]]]

# Type for color values including special colors and arbitrary values
ColorValue = Union[ColorSpec, str]

## Color Validation

Functions to validate and parse color values:

In [ ]:
#| export
def is_valid_color_family(
    value: str  # The value to check
) -> bool:  # True if value is a valid color family
    """Check if a value is a valid Tailwind color family."""
    return value in [c.value for c in ColorFamily]

In [ ]:
#| export
def is_valid_shade(
    value: str  # The value to check
) -> bool:  # True if value is a valid shade
    """Check if a value is a valid Tailwind shade."""
    return value in [s.value for s in ColorShade]

In [ ]:
#| export
def parse_color_spec(
    value: ColorSpec  # The color specification to parse
) -> tuple[str, Optional[str]]:  # Tuple of (color_family, shade) or (special_color, None)
    """
    Parse a color specification into family and shade.
    
    Examples:
    - "red-500" -> ("red", "500")
    - ("red", "500") -> ("red", "500")
    - "transparent" -> ("transparent", None)
    """
    if isinstance(value, tuple):
        color, shade = value
        return (color.value if isinstance(color, ColorFamily) else color,
                shade.value if isinstance(shade, ColorShade) else shade)
    
    if isinstance(value, str):
        # Check if it's a special color
        if value in SPECIAL_COLORS:
            return (value, None)
        
        # Try to parse as "color-shade"
        parts = value.split("-")
        if len(parts) == 2 and is_valid_color_family(parts[0]) and is_valid_shade(parts[1]):
            return (parts[0], parts[1])
    
    # Return as-is for arbitrary values
    return (value, None)

## Colored Utility Class

A utility class that supports Tailwind colors with opacity:

In [ ]:
#| export
class ColoredUtility(BaseUtility):
    """Utility class with color and opacity support."""
    
    def __init__(
        self,
        prefix: str,  # The utility prefix (e.g., 'bg', 'text', 'border')
        color: Optional[ColorValue] = None,  # The color value
        opacity: Optional[Union[int, str]] = None  # Optional opacity value (0-100 or arbitrary)
    ):
        """Initialize with prefix, optional color, and optional opacity."""
        super().__init__(prefix)
        self._color = color
        self._opacity = opacity
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format (color specification)
    ) -> str:  # The formatted color value
        """
        Format color value according to Tailwind conventions.
        Handles color-shade combinations, special colors, and arbitrary values.
        """
        color, shade = parse_color_spec(value)
        
        # Handle special colors
        if color in SPECIAL_COLORS:
            return color
        
        # Handle valid color-shade combinations
        if shade and is_valid_color_family(color) and is_valid_shade(shade):
            return f"{color}-{shade}"
        
        # Handle custom properties
        if is_custom_property(str(value)):
            return f"({value})"
        
        # Handle arbitrary values
        if is_arbitrary_value(str(value)) or not (is_valid_color_family(color) or color in SPECIAL_COLORS):
            return f"[{value}]"
        
        # Return as-is for other cases
        return str(value)
    
    def _build_class(
        self,
        value: Optional[ColorValue] = None  # Optional color value to use
    ) -> str:  # The complete CSS class string
        """Build the complete CSS class string with optional opacity."""
        # Use provided value or stored color
        color_value = value if value is not None else self._color
        
        if color_value is None:
            return self.prefix
        
        # Format the color part
        formatted_color = self._format_value(color_value)
        base_class = f"{self.prefix}-{formatted_color}"
        
        # Add opacity if specified
        if self._opacity is not None:
            base_class = f"{base_class}/{self._opacity}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class
    
    def opacity(
        self,
        value: Union[int, str]  # Opacity value (0-100 or arbitrary)
    ) -> 'ColoredUtility':  # A new instance with opacity set
        """Return a new ColoredUtility instance with opacity value."""
        # Create a new instance to avoid modifying the original
        new_instance = ColoredUtility(self.prefix, self._color, value)
        new_instance._modifiers = self._modifiers.copy() if self._modifiers else []
        new_instance._value = self._value
        return new_instance

## Colored Factory

Factory for creating color-based utilities with convenient API:

In [ ]:
#| export
class ColoredFactory(BaseFactory):
    """Factory for creating color-based utilities."""
    
    def __init__(
        self,
        prefix: str,  # The utility prefix (e.g., 'bg', 'text', 'border')
        doc: Optional[str] = None  # Optional documentation string
    ):
        """Initialize factory with prefix and documentation."""
        doc = doc or f"Factory for {prefix} color utilities"
        super().__init__(doc)
        self.prefix = prefix
        
        # Create sub-factories for each color family
        for color in ColorFamily:
            setattr(self, color.value, ColorFamilyProxy(self.prefix, color.value))
        
        # Create sub-factories for special colors
        for special in SPECIAL_COLORS:
            setattr(self, special, ColoredUtility(self.prefix, special))
    
    def __call__(
        self,
        color: ColorValue,  # The color specification
        opacity: Optional[Union[int, str]] = None  # Optional opacity
    ) -> ColoredUtility:  # A new ColoredUtility instance
        """Create a ColoredUtility with specified color and optional opacity."""
        utility = ColoredUtility(self.prefix, color, opacity)
        return utility
    
    def __getattr__(
        self,
        name: str  # Attribute name (special value or arbitrary)
    ) -> ColoredUtility:  # ColoredUtility for arbitrary values
        """
        Handle attribute access for arbitrary color values.
        Note: For standard colors, use the structured approach: bg.red._500
        """
        # Only handle arbitrary color values here
        return ColoredUtility(self.prefix, name.replace("_", "-"))
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get detailed information about this color factory."""
        color_families = [c.value for c in ColorFamily]
        shades = [s.value for s in ColorShade]
        
        return {
            'description': self._doc,
            'valid_inputs': {
                'color_families': color_families,
                'shades': shades,
                'special_colors': list(SPECIAL_COLORS.keys()),
                'usage_examples': [
                    f'{self.prefix}.red._500',
                    f'{self.prefix}.blue._300.opacity(75)',
                    f'{self.prefix}("red-500", opacity=50)',
                    f'{self.prefix}.transparent',
                    f'{self.prefix}("#ff0000")',
                    f'{self.prefix}("--custom-color")'
                ]
            },
            'options': {
                'prefix': self.prefix,
                'supports_opacity': True,
                'opacity_syntax': 'color/opacity'
            }
        }

## Color Family Proxy

A proxy class to handle color family attribute access (e.g., `bg.red.500`):

In [ ]:
#| export
class ColorFamilyProxy:
    """Proxy for accessing color shades via dot notation."""
    
    def __init__(
        self,
        prefix: str,  # The utility prefix
        color_family: str  # The color family name
    ):
        """Initialize with prefix and color family."""
        self.prefix = prefix
        self.color_family = color_family
        
        # Add shade properties for autocomplete
        self._50 = ColoredUtility(prefix, f"{color_family}-50")
        self._100 = ColoredUtility(prefix, f"{color_family}-100")
        self._200 = ColoredUtility(prefix, f"{color_family}-200")
        self._300 = ColoredUtility(prefix, f"{color_family}-300")
        self._400 = ColoredUtility(prefix, f"{color_family}-400")
        self._500 = ColoredUtility(prefix, f"{color_family}-500")
        self._600 = ColoredUtility(prefix, f"{color_family}-600")
        self._700 = ColoredUtility(prefix, f"{color_family}-700")
        self._800 = ColoredUtility(prefix, f"{color_family}-800")
        self._900 = ColoredUtility(prefix, f"{color_family}-900")
        self._950 = ColoredUtility(prefix, f"{color_family}-950")
    
    def __getattr__(
        self,
        shade: str  # The shade value (e.g., "500", "950")
    ) -> ColoredUtility:  # A ColoredUtility with the color-shade combination
        """
        Handle shade access.
        Example: bg.red.500 -> ColoredUtility("bg", "red-500")
        """
        # Handle underscore prefix for numeric shades (e.g., ._500)
        if shade.startswith("_"):
            shade = shade[1:]
        
        color_spec = f"{self.color_family}-{shade}"
        return ColoredUtility(self.prefix, color_spec)
    
    def __call__(
        self,
        shade: Union[ColorShade, str],  # The shade value
        opacity: Optional[Union[int, str]] = None  # Optional opacity
    ) -> ColoredUtility:  # A ColoredUtility instance
        """
        Create utility with shade and optional opacity.
        Example: bg.red("500", opacity=75)
        """
        shade_value = shade.value if isinstance(shade, ColorShade) else shade
        color_spec = f"{self.color_family}-{shade_value}"
        return ColoredUtility(self.prefix, color_spec, opacity)

## Examples

Test the color system with various use cases:

In [ ]:
# Test basic color usage
bg = ColoredFactory("bg", "Background color utilities")
print(bg.red._500)

# Test standard color-shade combinations
assert str(bg.red._500) == "bg-red-500"
assert str(bg.blue._300) == "bg-blue-300"
assert str(bg.green._950) == "bg-green-950"

bg-red-500


In [ ]:
# Test color family proxy access
assert str(bg.red._500) == "bg-red-500"
assert str(bg.blue._300) == "bg-blue-300"
assert str(bg.slate._950) == "bg-slate-950"

# Test with ColorShade enum
assert str(bg.red(ColorShade.SHADE_500)) == "bg-red-500"

In [ ]:
# Test special colors
assert str(bg.transparent) == "bg-transparent"
assert str(bg.black) == "bg-black"
assert str(bg.white) == "bg-white"
assert str(bg.current) == "bg-current"
assert str(bg.inherit) == "bg-inherit"

In [ ]:
# Test opacity modifiers
assert str(bg.red._500.opacity(50)) == "bg-red-500/50"
assert str(bg.blue._300.opacity(75)) == "bg-blue-300/75"
assert str(bg.black.opacity(10)) == "bg-black/10"

# Test opacity with arbitrary values
assert str(bg.red._500.opacity("[0.87]")) == "bg-red-500/[0.87]"

In [ ]:
# Test arbitrary color values
assert str(bg("#ff0000")) == "bg-[#ff0000]"
assert str(bg("rgb(255, 0, 0)")) == "bg-[rgb(255, 0, 0)]"
assert str(bg("hsl(0, 100%, 50%)")) == "bg-[hsl(0, 100%, 50%)]"

In [ ]:
# Test CSS custom properties
assert str(bg("--custom-bg")) == "bg-(--custom-bg)"
assert str(bg("--theme-primary")) == "bg-(--theme-primary)"

# Test with opacity
assert str(bg("--custom-bg", opacity=50)) == "bg-(--custom-bg)/50"

In [ ]:
# Test factory call syntax
assert str(bg("red-500")) == "bg-red-500"
assert str(bg("red-500", opacity=75)) == "bg-red-500/75"
assert str(bg((ColorFamily.BLUE, ColorShade.SHADE_300))) == "bg-blue-300"
assert str(bg((ColorFamily.GREEN, "500"))) == "bg-green-500"

## Test Functions

Comprehensive test functions following the project's naming convention:

In [ ]:
#| export
def test_colors_enum_examples():
    """Test color family and shade enums."""
    # Test ColorFamily enum
    assert ColorFamily.RED.value == "red"
    assert ColorFamily.BLUE.value == "blue"
    assert ColorFamily.SLATE.value == "slate"
    assert len(ColorFamily) == 22  # Verify we have all 22 color families
    
    # Test ColorShade enum
    assert ColorShade.SHADE_50.value == "50"
    assert ColorShade.SHADE_500.value == "500"
    assert ColorShade.SHADE_950.value == "950"
    assert len(ColorShade) == 11  # Verify we have all 11 shades

# Run the test
test_colors_enum_examples()

In [ ]:
#| export
def test_colors_validation_examples():
    """Test color validation functions."""
    # Test valid color families
    assert is_valid_color_family("red") == True
    assert is_valid_color_family("blue") == True
    assert is_valid_color_family("slate") == True
    assert is_valid_color_family("invalid") == False
    
    # Test valid shades
    assert is_valid_shade("50") == True
    assert is_valid_shade("500") == True
    assert is_valid_shade("950") == True
    assert is_valid_shade("999") == False
    
    # Test parse_color_spec
    assert parse_color_spec("red-500") == ("red", "500")
    assert parse_color_spec(("blue", "300")) == ("blue", "300")
    assert parse_color_spec("transparent") == ("transparent", None)
    assert parse_color_spec("#ff0000") == ("#ff0000", None)

# Run the test
test_colors_validation_examples()

In [ ]:
#| export
def test_colors_factory_examples():
    """Test ColoredFactory with various color specifications."""
    bg = ColoredFactory("bg")
    
    # Test standard colors with structured approach
    assert str(bg.red._500) == "bg-red-500"
    assert str(bg.blue._300) == "bg-blue-300"
    assert str(bg.green._950) == "bg-green-950"
    
    # Test special colors
    assert str(bg.transparent) == "bg-transparent"
    assert str(bg.black) == "bg-black"
    assert str(bg.white) == "bg-white"
    
    # Test with call syntax
    assert str(bg("red-500")) == "bg-red-500"
    assert str(bg("blue-300", opacity=50)) == "bg-blue-300/50"

# Run the test
test_colors_factory_examples()

In [ ]:
#| export
def test_colors_opacity_examples():
    """Test opacity modifiers with color utilities."""
    bg = ColoredFactory("bg")
    
    # Test opacity with standard colors
    assert str(bg.red._500.opacity(50)) == "bg-red-500/50"
    assert str(bg.blue._300.opacity(75)) == "bg-blue-300/75"
    assert str(bg.black.opacity(10)) == "bg-black/10"
    
    # Test opacity with arbitrary values
    assert str(bg.red._500.opacity("[0.87]")) == "bg-red-500/[0.87]"
    assert str(bg.blue._300.opacity("(--my-opacity)")) == "bg-blue-300/(--my-opacity)"
    
    # Test opacity in factory call
    assert str(bg("green-600", opacity=25)) == "bg-green-600/25"
    
    # Test that opacity doesn't persist (bug fix)
    red_500 = bg.red._500
    assert str(red_500) == "bg-red-500"
    assert str(red_500.opacity(50)) == "bg-red-500/50"
    assert str(red_500) == "bg-red-500"  # Should still be without opacity

# Run the test
test_colors_opacity_examples()

In [ ]:
#| export
def test_colors_arbitrary_examples():
    """Test arbitrary color values and custom properties."""
    bg = ColoredFactory("bg")
    
    # Test hex colors
    assert str(bg("#ff0000")) == "bg-[#ff0000]"
    assert str(bg("#00ff00")) == "bg-[#00ff00]"
    assert str(bg("#0000ff")) == "bg-[#0000ff]"
    
    # Test RGB/HSL functions
    assert str(bg("rgb(255, 0, 0)")) == "bg-[rgb(255, 0, 0)]"
    assert str(bg("hsl(0, 100%, 50%)")) == "bg-[hsl(0, 100%, 50%)]"
    assert str(bg("rgba(255, 0, 0, 0.5)")) == "bg-[rgba(255, 0, 0, 0.5)]"
    
    # Test CSS custom properties
    assert str(bg("--custom-bg")) == "bg-(--custom-bg)"
    assert str(bg("--theme-primary")) == "bg-(--theme-primary)"

# Run the test
test_colors_arbitrary_examples()

In [ ]:
#| export
def test_colors_proxy_examples():
    """Test ColorFamilyProxy for dot notation access."""
    bg = ColoredFactory("bg")
    
    # Test direct shade access
    assert str(bg.red._50) == "bg-red-50"
    assert str(bg.red._500) == "bg-red-500"
    assert str(bg.red._950) == "bg-red-950"
    
    # Test all shade attributes are available for autocomplete
    assert hasattr(bg.red, '_50')
    assert hasattr(bg.red, '_100')
    assert hasattr(bg.red, '_200')
    assert hasattr(bg.red, '_300')
    assert hasattr(bg.red, '_400')
    assert hasattr(bg.red, '_500')
    assert hasattr(bg.red, '_600')
    assert hasattr(bg.red, '_700')
    assert hasattr(bg.red, '_800')
    assert hasattr(bg.red, '_900')
    assert hasattr(bg.red, '_950')
    
    # Verify they are ColoredUtility instances
    assert isinstance(bg.blue._300, ColoredUtility)
    assert isinstance(bg.green._600, ColoredUtility)
    
    # Test call syntax on proxy
    assert str(bg.blue("300")) == "bg-blue-300"
    assert str(bg.green(ColorShade.SHADE_600)) == "bg-green-600"
    assert str(bg.purple("400", opacity=60)) == "bg-purple-400/60"

# Run the test
test_colors_proxy_examples()

## Practical Examples

Test with multiple utility prefixes to show the color system is reusable:

In [ ]:
#| export
def test_colors_multiple_utilities_examples():
    """Test color system with multiple utility types."""
    # Create factories for different utilities
    bg = ColoredFactory("bg", "Background color utilities")
    text = ColoredFactory("text", "Text color utilities")
    border = ColoredFactory("border", "Border color utilities")
    
    # Test same color across different utilities
    assert str(bg.red._500) == "bg-red-500"
    assert str(text.red._500) == "text-red-500"
    assert str(border.red._500) == "border-red-500"
    
    # Test with opacity
    assert str(bg.blue._300.opacity(50)) == "bg-blue-300/50"
    assert str(text.blue._300.opacity(50)) == "text-blue-300/50"
    assert str(border.blue._300.opacity(50)) == "border-blue-300/50"
    
    # Test special colors
    assert str(bg.transparent) == "bg-transparent"
    assert str(text.transparent) == "text-transparent"
    assert str(border.transparent) == "border-transparent"

# Run the test
test_colors_multiple_utilities_examples()

In [ ]:
#| export
def test_colors_practical_usage_examples():
    """Test practical usage patterns with FastHTML components."""
    from fasthtml.common import Div, Span, P
    from cjm_fasthtml_tailwind.core.base import combine_classes
    
    bg = ColoredFactory("bg")
    text = ColoredFactory("text")
    border = ColoredFactory("border")
    
    # Create a colored card component
    card = Div(
        P("Hello World", cls=str(text.gray._700)),
        cls=combine_classes(
            bg.white,
            border.gray._200,
            "rounded-lg p-4 border"
        )
    )
    
    # Verify classes
    assert "bg-white" in card.attrs['class']
    assert "border-gray-200" in card.attrs['class']
    assert card.children[0].attrs['class'] == "text-gray-700"
    
    # Create a semi-transparent overlay
    overlay = Div(
        cls=str(bg.black.opacity(50))
    )
    assert overlay.attrs['class'] == "bg-black/50"
    
    # Create a component with custom color
    custom = Span(
        "Custom",
        cls=combine_classes(
            bg("--theme-primary"),
            text.white
        )
    )
    assert "bg-(--theme-primary)" in custom.attrs['class']
    assert "text-white" in custom.attrs['class']

# Run the test
test_colors_practical_usage_examples()

## Helper Functions

Utility functions for working with colors:

In [ ]:
#| export
def get_all_color_families() -> List[str]:
    """Get list of all Tailwind color family names."""
    return [c.value for c in ColorFamily]

In [ ]:
#| export
def get_all_shades() -> List[str]:
    """Get list of all Tailwind shade values."""
    return [s.value for s in ColorShade]

In [ ]:
#| export
def get_all_color_specs() -> List[str]:
    """Get list of all valid color-shade combinations."""
    specs = []
    for family in ColorFamily:
        for shade in ColorShade:
            specs.append(f"{family.value}-{shade.value}")
    return specs

In [ ]:
# Test helper functions
families = get_all_color_families()
assert len(families) == 22
assert "red" in families
assert "blue" in families
assert "slate" in families

shades = get_all_shades()
assert len(shades) == 11
assert "50" in shades
assert "500" in shades
assert "950" in shades

specs = get_all_color_specs()
assert len(specs) == 22 * 11  # 242 combinations
assert "red-500" in specs
assert "blue-300" in specs
assert "slate-950" in specs

print("✅ All color helper functions work correctly!")

✅ All color helper functions work correctly!


## Documentation Test

Test that the factory provides proper documentation:

In [ ]:
#| export
def test_colors_factory_documentation():
    """Test that color factories have proper documentation."""
    bg = ColoredFactory("bg", "Background color utilities")
    text = ColoredFactory("text", "Text color utilities")
    
    # Test describe method
    assert bg.describe() == "Background color utilities"
    assert text.describe() == "Text color utilities"
    
    # Test get_info method
    info = bg.get_info()
    assert info['description'] == "Background color utilities"
    assert 'color_families' in info['valid_inputs']
    assert 'shades' in info['valid_inputs']
    assert 'special_colors' in info['valid_inputs']
    assert len(info['valid_inputs']['color_families']) == 22
    assert len(info['valid_inputs']['shades']) == 11
    assert info['options']['prefix'] == 'bg'
    assert info['options']['supports_opacity'] == True

# Run the test
test_colors_factory_documentation()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()